In [4]:
import numpy as np
import os
import utils
import time
#from extractDigitFeatures import extractDigitFeatures
#from trainModel import trainModel
from evaluateLabels import evaluateLabels
from evaluateModel import evaluateModel

In [17]:
# There are three versions of MNIST dataset
dataTypes = ['digits-jitter.mat'] #'digits-normal.mat', 'digits-scaled.mat', 

# You have to implement three types of features
featureTypes = ['pixel', 'hog', 'lbp'] # 

# Accuracy placeholder
accuracy_mat = np.zeros((len(dataTypes), len(featureTypes)))
val_accuracy = np.zeros((len(dataTypes), len(featureTypes)))
trainSet = 1
testSet = 2
validationSet = 3

In [20]:
for i in range(len(dataTypes)):
    dataType = dataTypes[i]
    #Load data
    path = os.path.join('../..', 'data', dataType)
    data = utils.loadmat(path)
    print ('+++ Loading digits of dataType: {}'.format(dataType))

    # Optionally montage the digits in the val set
    #montageDigits(data['x'][:, :, data['set']==2])

    for j in range(len(featureTypes)):
        featureType = featureTypes[j]

        # Extract features
        tic = time.time()
        features = extractDigitFeatures(data['x'], featureType, dataType)
        print ('{:.2f}s to extract {} features for {} images'.format(time.time()-tic,
                featureType, features.shape[1]))

        # Train model
        tic = time.time()
        model = trainModel(features[:, data['set']==trainSet], data['y'][data['set']==trainSet], 
                          features[:, data['set']==3], data['y'][data['set']==validationSet])
        print ('{:.2f}s to train model'.format(time.time()-tic))
        print ('Accuracy [validationSet={}] {:.2f}\n'.format(validationSet, model['lastAccuracy']))

        # Test the model
        ypred = evaluateModel(model, features[:, data['set']==testSet])
        y = data['y'][data['set']==testSet]

        # Measure accuracy
        (acc, conf) = evaluateLabels(y, ypred, False)
        print ('Accuracy [testSet={}] {:.2f}\n'.format(testSet, acc*100))
        accuracy_mat[i, j] = acc
        val_accuracy[i, j] = model['lastAccuracy']

+++ Loading digits of dataType: digits-jitter.mat
2.79s to extract pixel features for 2000 images
18.99s to train model
Accuracy [validationSet=3] 0.14

Accuracy [testSet=2] 15.80

9.14s to extract hog features for 2000 images
73.34s to train model
Accuracy [validationSet=3] 0.10

Accuracy [testSet=2] 10.00

25.76s to extract lbp features for 2000 images
3.06s to train model
Accuracy [validationSet=3] 0.56

Accuracy [testSet=2] 59.80



In [ ]:
# Print the results in a table
print ('+++ Accuracy Table [trainSet={}, validationSet = {}, testSet={}]'.format(trainSet, validationSet, testSet))
print ('--------------------------------------------------')
print ('dataset\t\t\t',)
for j in range(len(featureTypes)):
    print ('{}\t'.format(featureTypes[j]),)

print ('')
print ('--------------------------------------------------')
for i in range(len(dataTypes)):
    print ('{}\t'.format(dataTypes[i]),)
    for j in range(len(featureTypes)):
        print ('{:.2f}\t'.format(accuracy[i, j]*100), '{:.2f}\t'.format(val_accuracy[i, j]*100))
    print ('')

In [2]:
import numpy as np
import utils
from utils import loadmat
from joblib import Parallel, delayed
import multiprocessing

# EXTRACTDIGITFEATURES extracts features from digit images
#   features = extractDigitFeatures(x, featureType) extracts FEATURES from images
#   images X of the provided FEATURETYPE. The images are assumed to the of
#   size [W H 1 N] where the first two dimensions are the width and height.
#   The output is of size [D N] where D is the size of each feature and N
#   is the number of images. 

def pixelFeatures(x):
    return feature_normalization(x.reshape(-1), 'min-max')

def feature_normalization(patch, type, epsilon=1e-5):
    patch = patch.astype('longdouble')
    if type == 'Sqrt':
        return np.sqrt(patch)
    elif type == 'L2-Norm':
        return patch / np.sqrt(np.sum(patch ** 2) + epsilon)
    elif type == 'min-max':
        return (patch - patch.min()) / (patch.max() - patch.min())
    else:
        return -1
    
def extract_relevant_window(arr, patch_size, index_i, index_j):
    curr_arr = np.zeros((patch_size, patch_size))
    for i in range(patch_size):
        for j in range(patch_size):
            curr_arr[i][j] = arr[index_j*patch_size + i][index_i*patch_size + j]     
    return curr_arr

def hogFeatures(x):
    # Applying Non Linear Mapping
    img = feature_normalization (x, 'Sqrt')

    # Computing the channel gradient
    r_grad, c_grad = np.empty(img.shape).astype('longdouble'), np.empty(img.shape).astype('longdouble')
    
    r_grad[1:-1,] = img[2:, :] - img[:-2, :] 
    c_grad[:, 1:-1] = img[:, 2:] - img[:, :-2]
    c_grad[:, 0], c_grad[:, -1] = 0, 0
    r_grad[0, :], r_grad[-1, 0] = 0, 0

    img_magnitude = np.sqrt(np.power(r_grad, 2) + np.power(c_grad, 2))
    img_theta = np.rad2deg(np.arctan(c_grad/(r_grad+0.00000001))) % 180
    orientation_bins = 8
    patch_size = 4
    tot_r, tot_c = img.shape
    hog = np.zeros((int(tot_r/patch_size), int(tot_c/patch_size), orientation_bins))
    for j in range(int(tot_c/patch_size)):
        for i in range(int(tot_r/patch_size)):
            # Extract the Current Patch and weight
            curr_patch = extract_relevant_window(img_theta, patch_size, i, j)
            curr_weight = extract_relevant_window(img_magnitude, patch_size, i, j)
            # Applying Histogram calculations
            hog[j][i] = np.histogram(np.ndarray.flatten(curr_patch), weights=np.ndarray.flatten(curr_weight), 
                                     bins=np.linspace(0, 180, num=(orientation_bins+1)))[0]        
    hog_norm = feature_normalization(hog, 'min-max')
    return hog_norm.ravel()

def compute_from_patch(patch):
    if patch.shape[0] != 3 and patch.shape[0] != patch.shape[1]:
        raise ValueError('Patch Size Mismatch')
    patch_sub = patch - np.ravel(patch)[4]
    patch_sub[patch_sub>0] = 1
    patch_sub[patch_sub<=0] = 0
    flattened_patch = np.delete(np.ravel(patch_sub), 4)
    return flattened_patch.dot(2**np.arange(flattened_patch.size)[::1])

def lbpFeatures(x):
    patch_size = 3
    img = np.sqrt(x)
    final_img = np.empty((img.shape[0]-(patch_size-1), img.shape[1]-(patch_size-1))).astype('longdouble')
    for r in range(0, final_img.shape[0]):
        for c in range(0, final_img.shape[1]):
            final_img[r][c] = compute_from_patch(img[r:r+patch_size, c:c+patch_size])
    return feature_normalization(np.histogram(np.ndarray.flatten(final_img), bins=np.linspace(1, 255, num=257))[0], 'min-max')

def extractDigitFeatures(x, featureType, dataType):
    N = x.shape[2]
    
    if featureType == 'pixel':
        features = np.empty((784, N)).astype('longdouble')
        features = np.array(Parallel(n_jobs=multiprocessing.cpu_count()) 
                           (delayed(pixelFeatures)(x[:, :, X_idx]) for X_idx in range(N)))
    elif featureType == 'hog':
        if dataType == 'digits-scaled.mat':
            x = feature_normalization(x, 'L2-Norm')
        features = np.empty((392, N)).astype('longdouble')
        features = np.array(Parallel(n_jobs=multiprocessing.cpu_count()) 
                           (delayed(hogFeatures)(x[:, :, X_idx]) for X_idx in range(N)))
    elif featureType == 'lbp':
        features = np.empty((256, N)).astype('longdouble')
        features = np.array(Parallel(n_jobs=multiprocessing.cpu_count()) 
                           (delayed(lbpFeatures)(x[:, :, X_idx]) for X_idx in range(N)))
        
    if featureType in ['lbp', 'pixel'] or dataType in ['digits-scaled.mat']:
        features = feature_normalization(features, 'Sqrt')
    return features.T

In [19]:
import numpy as np

def softmax(x):
    exp = np.exp(x - np.max(x))
    return exp / np.array([np.sum(exp, axis=1)]).T

def confusion_matrix(model, y):
        confusion_matrix = np.zeros((model['numClass'], model['numClass']))
        for itr in range(model['y_pred'].size):
            confusion_matrix[y[itr], model['y_pred'][itr]] += 1
        return confusion_matrix

def accuracy(model, X, y):
        y_pred = np.argmax(softmax(np.dot(X.T, model['w']) + model['b']), axis=1)
        model['y_pred'] = y_pred
        accuracy = np.sum([y_pred[i] == y[i] for i in range(y.size)]) / y.size
        return model, accuracy

def multiclassLRTrain(x, y, param):

    classLabels = np.unique(y)
    numClass = classLabels.shape[0]
    numFeats = x.shape[0]
    numData = x.shape[1]
    

    # Initialize weights randomly (Implement gradient descent)
    model = {}
    model['classLabels'] = classLabels
    model['numClass'] = numClass
    model['w'] = np.random.uniform(low=-0.01, high=.01, size=(x.shape[0], numClass))
    model['b'] = np.random.uniform(low=-0.01, high=.01)
    model['lastAccuracy'] = -1
    model['debug'] = param['debug']
    model['maxTol'] = param['maxTol']
    y_categorical = np.eye(model['numClass'])[y] 
    curTol = 0
        
    for itr in range(param['maxiter']):
        prediction = softmax(np.dot(x.T, model['w']))
        error = y_categorical - prediction
        gradient = np.dot(x, error)
        model['w'] += param['eta'] * (gradient - (param['lambda'] * model['w']))
        model['b'] -= param['eta'] * np.sum(error)

        model, curr_accuracy = accuracy(model, param['x_val'], param['y_val'])
        if curr_accuracy != model['lastAccuracy']:
            model['lastAccuracy'] = curr_accuracy
        else:
            curTol += 1
            if curTol >= model['maxTol']:
                return model
    return model

def trainModel(x, y, x_val, y_val):
    param = {}
    param['lambda'] = 0.01      # Regularization term
    param['maxiter'] = 20000     # Number of iterations
    param['eta'] = 0.001         # Learning rate
    param['x_val'] = x_val
    param['y_val'] = y_val
    param['debug'] = False
    param['maxTol'] = 15
    
    return multiclassLRTrain(x, y, param)